# Itereren over een dataframe

In principe zelfde idee als gewone for
´´´
for rij in df:
    # doe iets met de rij
´´´

maar dit itereert over de namen van de kolommen.

Om over de rijen te itereren moet je **iterrows()** gebruiken.

Die moet je op volgende manier gebruiken:


### Opgepast!

Dingen veranderen in de variabele rij heeft géén blijvend effect!
*rij* is een kopie van de rij en je verandert dus alleen de kopie.

Juiste aanpak = aanpassingen doen via het dataframe zelf.

Voorbeeld: *df.loc[index, “dubbele_index"] = index \* 2*

Dit is o.a. nuttig in oefening 8.1.1.1.1.


## Enkele use-cases met voorbeelden

### Use case 1: iets 'doen' met elke rij van het dataframe

#### Voorbeeld 1a: informatie leesbaar afdrukken

In [ ]:
import pandas as pd

def maakDataFrame():
    df = pd.DataFrame()
    df["voornaam"] = ["Jan","Petra","Joris","Kornelia","Ali", "Sefia"]
    df["leeftijd"] = [19, 24, 12, 18, 22, 17]
    df["geslacht"] = ["M","V","M","V","M","V"]
    return df

def printPersoonInfo(df):    
    for index, rij in df.iterrows():
        print("Persoon", index+1, ":", rij.voornaam, "is", rij.leeftijd, "jaar.")

In [ ]:
zesPersonen = maakDataFrame()
printPersoonInfo(zesPersonen)

**Opgelet**: wanneer je informatie in het dataframe wil aanpassen, mag je NIET via de variabele *rij* werken, maar moet je dit rechtstreeks in het dataframe doen met **.loc**.

Disclaimer: onderstaand voorbeeld kan je ook met een apply doen, maar deze aanpak is wél nodig in een aantal oefeningen van zo dadelijk.

In [ ]:
def voegLevensverwachtingToe(df):
    # in België is bij de geboorte de levensverwachting voor mannen 79,5 jaar; bij vrouwen 83,8.
    # omdat jullie nog in leven zijn en het gemiddelde moeten optrekken, is dit iets meer,
    # maar die cijfers hebben we niet.
    for index, rij in df.iterrows():
        if rij.geslacht == "M":
            verwachting = 79.5 - rij.leeftijd
        else:
            verwachting = 83.8 - rij.leeftijd
        df.loc[index,"levensverwachting"] = verwachting
        
zesPersonen = maakDataFrame()
print(zesPersonen)
voegLevensverwachtingToe(zesPersonen)
print(zesPersonen)

## Voorbeeld 1b: een cumulatieve berekening die anders is dan cumsum of cumproduct

In dit voorbeeld willen we nagaan hoeveel water er op een plein blijft staan tijdens en kort na een hevige bui. Op https://www.vmm.be/water/bouwen/afvalwater/ontwerp-van-de-huisriolering vind je meer info over je een afvoerbuis moet dimensioneren, maar we veronderstellen hierbij een buis van 125mm doormeter en een helling van 5mm per m. De afvoercapaciteit van zo'n buis is 24,6 m³/uur.
Verder houden we rekening met de neerslaggegevens van https://klimaatadaptatienederland.nl/stresstest/bijsluiter/wateroverlast/basisinformatie/waterdiepte-kortdurende-hevige-neerslag/ waar we lezen dat de kans op een bui van 70 mm op 2 uur een kans heeft van 1/100 per jaar, en een bui van 140 mm op 2 uur een kan van 1/1000 per jaar. Per vierkante meter oppervlakte is dat dan respectievelijk 35 en 70 liter per uur. 

Volgende functie rekent dan uit hoeveel water er een op een plein van 40 x 20 m² zal staan met één afvoerbuis en een ondergrondse waterput van 5000 liter als buffer. De invoer voor de functie is een dataframe met de hoeveelheid regen in mm per kwartier.

## Use case 2: Verbanden tussen rijen van een dataframe

Basisidee: apply is heel handig om functie los te laten op één rij.
Dit is wel zonder context van voorgaande of volgende rij.
In veel gevallen wil je wél verbanden tussen rijen, zoals
- Nagaan hoeveel keren de rij stijgt, 
- Berekenen wat de procentuele stijging is tussen opeenvolgende rijen 
- Controleren of er waarden uit de band springen
- … 

Opzet om dit wél te kunnen doen:
- Gebruik iterrows om doorheen het dataframe te gaan
- Gebruik variabele *vorige* om vorige rij bij te houden

Deze extra variabele *vorige*  moeten we vòòr de for-lus initialiseren en op het einde van elke passage door de lus aanpassen zodat die mee ‘opschuift’.  

Dat geeft volgend aangepast schema: 

In [ ]:
def waterOpHetPlein(file = "../csv/regen.csv"):
    df = pd.read_csv(file, sep=";")
    water = 0
    for index, rij in df.iterrows():
        potentieleAfvoer = 24.6 * 1000 / 4
        toevoer = rij.regen * 40 * 20
        water = water - potentieleAfvoer + toevoer
        if water < 0:
            water = 0
        waterOpPlein = max(water - 5000, 0)
        # buffervulling
        df.loc[index, "buffervulling"] = min(water, 5000)
        df.loc[index, "waterOpPlein"] = waterOpPlein
    return df

print(waterOpHetPlein())
print(waterOpHetPlein("../csv/regen2.csv"))



### Voorbeeld: hoe dikwijls zakte het indexcijfer?

In [ ]:
def aantalDalingen(csvbestand):
    df = pd.read_csv(csvbestand, sep=";", encoding="latin-1")
    aantal = 0
    vorige = df.loc[0, "indexcijfer"]
    for index, rij in df.iterrows():
        if vorige > rij.indexcijfer:
            aantal = aantal + 1
        vorige = rij.indexcijfer
    return aantal

aantalKerenGedaald = aantalDalingen("../csv/consumptie_index_september.csv")
print("Consumptieindex is", aantalKerenGedaald, "keren gedaald.")

## Oefeningen op iterrows

1. Zij gegeven een dataframe punten.csv met kolommen naam, eersteKans, tweedeKans voor de 
punten van een vak. Print voor elke naam de hoogste score van de eerste en tweede kans. 
*Dit csv-bestand moet je inlezen zonder encoding.* 

In [ ]:
import pandas as pd
def printPunten(file = "../csv/punten.csv"):
    df = pd.read_csv(file, sep=";")
    for index, rij in df.iterrows():
        print(rij.naam, "behaalde", max(rij.eersteKans, rij.tweedeKans))
        
printPunten()

2. Zij gegeven een dataframe mensen.csv met lengte in feet en gewicht in pounds. Zet de imperial 
gegevens om naar cm en kg. 1 voet is 30,48 cm en 1 pound is 0,4536 kg. Rond af tot één cijfer na de komma met de functie round. 
*Dit csv-bestand moet je inlezen zonder encoding.* 

In [ ]:
def imperialToMetric(file ="../csv/mensen.csv"):
    df = pd.read_csv(file, sep=";")
    for index, rij in df.iterrows():
        gewicht = round(rij.gewicht * 0.4536, 1)
        lengte = round(rij.lengte * 30.48, 1) 
        df.loc[index,"gewicht"] = gewicht
        df.loc[index,"lengte"] = lengte
    df.to_csv("../csv/mensenMetric.csv",sep=";", encoding="latin-1")
    
imperialToMetric()

6. In het csv-bestand csv/diepvries.csv vind je de kostprijs van een aantal diepvriezers en hun jaarlijks verbruik. In deze csv staat ook de huidige diepvriezer die ‘gratis’ is, maar wel een hoog verbruik kent. Bereken per diepvries de gecumuleerde kostprijs na 10 en na 20 jaar bij een energieprijs van 34 eurocent/kWh. Voeg beide kolommen toe aan het dataframe.

In [ ]:
def simuleerKostprijsDiepvries(aankoopprijs, verbruik, prijsKWhCent, jaar):
    return aankoopprijs + verbruik*prijsKWhCent/100*jaar

def simuleerDiepvries():
    df = pd.read_csv("../csv/diepvries.csv", sep=";", encoding="latin-1")
    for index, rij in df.iterrows():
        verbruik = rij.verbruik
        aankoopprijs = rij.aankoopprijs
        prijs = simuleerKostprijsDiepvries(aankoopprijs, verbruik, 34, 10)
        df.loc[index,"jaar10"] = prijs
        prijs = simuleerKostprijsDiepvries(aankoopprijs, verbruik, 34, 20)
        df.loc[index,"jaar20"] = prijs
    df.to_csv("../csv/diepvriesBerekend.csv", sep=";", encoding="latin-1")
    return df
uitgebreidDf = simuleerDiepvries()
print(uitgebreidDf)